# INTEGRACIÓN PARA CONSTRUCCIÓN DE UN GOLDEN DATASET CON API GPT-3.5 Turbo

In [4]:
import pandas as pd
import pathlib
import random
import os
import ast

from openai import OpenAI
from dotenv import load_dotenv
from collections import defaultdict

#pip install --upgrade openai==1.1.1
#pip install cohere tiktoken
#!pip3 install python-dotenv

#print(openai.__version__)

In [6]:
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
# Utiliza la función to_excel para exportar el DataFrame a un archivo Excel
df = pd.read_excel(ruta_archivo)

In [7]:
df

,acronyms,expanded_acronyms,text,detected_acronyms,LLM_expanded_acronyms
0,ONG,ONG,Las ONG son organizaciones no gubernamentales ...,NaN,NaN
1,"CIA, FBI","CIA, FBI",La CIA y el FBI colaboran en investigaciones i...,NaN,NaN
2,"UE, OTAN","UE, OTAN",La unión europea y la OTAN tienen roles crucia...,NaN,NaN
3,NASA,NASA,"La NASA es la agencia espacial estadounidense,...",NaN,NaN
4,UNICEF,UNICEF,"UNICEF, trabajan por los derechos de los niños...",NaN,NaN
...,...,...,...,...,...
63,FAQ,FAQ,La sección de preguntas frecuentes FAQ proporc...,NaN,NaN
64,CEO,CEO,El CEO de la compañía anunció planes de expans...,NaN,NaN
65,CFO,CFO,El chief financial officer presentó el informe...,NaN,NaN
66,CTO,CTO,El CTO lidera el equipo de desarrollo tecnológ...,NaN,NaN


## Integración para acronyms con DSPY

In [ ]:
#!pip install git+https://github.com/stanfordnlp/dspy.git

In [1]:
#!pip list

In [ ]:
#!pip install --upgrade pydantic
#!pip install --upgrade openai

In [103]:
from typing import List, Union
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import numpy as np
import pickle
import chromadb
import random
import os
import re
from langchain.embeddings import HuggingFaceEmbeddings 

import dspy
from dotenv import load_dotenv

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
# Clase para dar formato al output del terminal
class Colors:
    BLUE = '\033[94m'
    ENDC = '\033[0m'

In [33]:
###########
#   LLM   #
###########
path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8090, url="http://127.0.0.0")
#lm = dspy.OpenAI(model="gpt-3.5-turbo")# "gpt-4o-2024-05-13")

dspy.settings.configure(lm=lm, temperature = 0)

/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env


### Just checking that DSPY with LLAMA works

In [15]:
"""
qa = dspy.ChainOfThought('question -> answer')

# Run with Llama instead
with dspy.context(lm=lm):
    response = qa(question="What is the first day of the week?")
    print('Model:', response.answer)
"""

Model: Sunday


### Synthetic acronymn generation

In [115]:
# Cargar dataframe donde se guardará la información 
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
df = pd.read_excel(ruta_archivo)
df.head(2)

,acronyms,expanded_acronyms,text,detected_acronyms,LLM_expanded_acronyms
0,ONG,Organización no gubernamental,NaN,NaN,NaN
1,PCAP,Pliego de cláusulas administrativas particulares,NaN,NaN,NaN


In [140]:
# Te he modificado esto un poco. Si haces las instrucciones en inglés, las definiciones del input y del output también deberían estar en inglés.
# Ten en cuenta que la signature es tu prompt, simplemente la estás definiendo como una clase
class AcronymGenerator(dspy.Signature):
    """
    Genera un texto en ESPAÑOL que contenga ACRONIMOS dentro de un contexto realista. 
    
    ACRONIMOS: IVA, PYME, IRPF, PIB
    TEXTO_GENERADO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
    afectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento
    de la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha
    ajustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,
    el PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.
    Sin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.
    """
    
    ACRONIMOS = dspy.InputField(desc="Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusión de varias otras palabras")
    TEXTO_GENERADO = dspy.OutputField(desc="Ejemplo de texto en español que incorpora los acrónimos en un contexto lo más realista posible")
    
class AcronymGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(AcronymGenerator)

    def forward(self, acronyms):  
        response = self.generator(ACRONIMOS=acronyms)    
        return dspy.Prediction(TEXTO_GENERADO=response.TEXTO_GENERADO)

In [119]:
lista_acronyms = df['acronyms'].str.lower().tolist()
lista_expanded_acronyms = df['expanded_acronyms'].str.lower().tolist()
# Crear una lista de tuplas con acrónimos y sus formas expandidas
acronym_pairs= tuple(zip(lista_acronyms, lista_expanded_acronyms))

In [118]:
lista_acronyms[0:3]

['ong', 'pcap', 'otan']

In [141]:
acronym_pairs[0:3]

(('ong', 'organización no gubernamental'),
 ('pcap', 'pliego de cláusulas administrativas particulares'),
 ('otan', 'organización del tratado del atlántico norte'))

In [142]:
acronyms = []

# Iterar sobre la lista de tuplas
for pair in acronym_pairs:
    # Para seleccionar aleatoriamente entre acrónimo o forma expandida
    selected = random.choice(pair)
    acronyms.append(selected)

# Lista con los elegidos
print(acronyms)

['organización no gubernamental', 'pliego de cláusulas administrativas particulares', 'otan', 'gigaherzio', 'fondo de las naciones unidas para la infancia', 'unión europea', 'fmi', 'oms', 'ceip', 'avenida', 'ffcc', 'inspección técnica de vehículos', 'televisión española', 's.a.', 'vp', 'punto kilométrico', 'pequeña y mediana empresa', 'bme', 'ccaa', 'cnmv', 'minhac', 'servicio publico de empleo estatal', 'once', 'ocde', 'boe', 'aena', 'renfe', 'radio nacional españa', 'rrhh']


In [144]:
# Lista para almacenar los textos generados
generated_texts = []

my_acronym_generator = AcronymGeneratorModule()
for ac in acronyms:
    print(ac)
    prediction = my_acronym_generator(ac)
    print(prediction)
    generated_texts.append(prediction.TEXTO_GENERADO)

organización no gubernamental
Prediction(
    TEXTO_GENERADO='La ONG ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. La organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. Gracias al apoyo de voluntarios y donantes, la ONG ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. Su labor ha sido fundamental para sensibilizar a la población sobre la urgencia de tomar medidas para preservar nuestro entorno natural.'
)
pliego de cláusulas administrativas particulares
Prediction(
    TEXTO_GENERADO='El contrato de obra pública se regirá por el pliego de cláusulas administrativas particulares (PCAP), el cual establece las condiciones específicas que deben cumplir tanto la administración como la empresa contratista. Es fundamental revisar detenidamente el PCAP para garantizar el cumplimiento de todas las oblig

Prediction(
    TEXTO_GENERADO='Las diferentes ccaa han implementado medidas para contener la propagación del virus, adaptándose a las necesidades específicas de cada región.'
)
cnmv
Prediction(
    TEXTO_GENERADO='La Comisión Nacional del Mercado de Valores (CNMV) es el organismo encargado de supervisar e inspeccionar los mercados de valores en España. Su principal objetivo es garantizar la transparencia y la protección de los inversores, velando por el correcto funcionamiento de los mercados financieros. La CNMV se encarga de regular la actividad de las entidades financieras y de velar por el cumplimiento de la normativa vigente en materia de valores. Su labor es fundamental para mantener la confianza en el sistema financiero y para prevenir posibles abusos o fraudes en el mercado de valores.'
)
minhac
Prediction(
    TEXTO_GENERADO='El Ministerio de Hacienda y Administraciones Públicas (MINHAC) ha anunciado nuevas medidas para mejorar la eficiencia en la gestión de los recursos públ

In [145]:
# Guardar en la columna 'text' del dataframe
df['text'] = generated_texts

In [155]:
#df.to_parquet('/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.parquet')

In [154]:
lm.inspect_history(4)




Genera un texto en ESPAÑOL que contenga ACRONIMOS dentro de un contexto realista. 

ACRONIMOS: IVA, PYME, IRPF, PIB
TEXTO_GENERADO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
afectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento
de la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha
ajustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,
el PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.
Sin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.

---

Follow the following format.

ACRONIMOS: Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusión de varias otras palabras
TEXTO GENERADO: Ejemplo de texto en español que incorpora los acrónimos 

'\n\n\nGenera un texto en ESPAÑOL que contenga ACRONIMOS dentro de un contexto realista. \n\nACRONIMOS: IVA, PYME, IRPF, PIB\nTEXTO_GENERADO: La reforma del sistema tributario ha traído cambios significativos en el IVA,\nafectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento\nde la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha\najustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,\nel PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.\nSin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.\n\n---\n\nFollow the following format.\n\nACRONIMOS: Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusión de varias otras palabras\nTEXTO GENERADO: Ejemplo de texto en español que incor

In [32]:
# Para comprobar la conexión al servidor LLaMA
!curl -X POST http://127.0.0.1:8090/generate -d '{"prompt": "Hello"}'

Expected request with `Content-Type: application/json`

In [ ]:
'''
filtered_samples = samples_train[samples_train['answers'].apply(lambda x: len(x) > 0)]
df_unique_titles = filtered_samples.drop_duplicates(subset='title', keep='first')
df_unique_titles.reset_index(drop=True, inplace=True)

new_samples = []
for idx, el in df_unique_titles.iterrows():
    if idx % 100 == 0:
        print(f"-- -- Processing index {idx} out of {len(df_unique_titles)}")
    cherry_picker = negator(question=el.question, golden_answer=el.answers[0]["text"])
    new_samples.append(
        [
            el.question, # question
            el.answers[0]["text"], # answer1
            cherry_picker.CHERRY_ANSWER, # answer2
            "CONTRADICTION", # label
            cherry_picker.RATIONALE, # rationale
        ]
    )

# Convert new_samples to a DataFrame if needed
new_samples_df = pd.DataFrame(new_samples, columns=['question', 'answer1', 'answer2', 'label', 'rationale'])
'''

In [157]:
path = '/export/usuarios_ml4ds/cggamella/RAG_tool/acronyms.parquet'
df_acron = pd.read_parquet(path)

# Detección de acrónimos con dspy  (Etapa 1)

In [5]:
# Cargar dataframe donde se guardará la información 
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
df = pd.read_excel(ruta_archivo)

In [18]:
df.columns

Index(['acronyms', 'expanded_acronyms', 'text', 'detected_acronyms',
       'LLM_expanded_acronyms'],
      dtype='object')

In [19]:
df.head(2)

,acronyms,expanded_acronyms,text,detected_acronyms,LLM_expanded_acronyms
0,ONG,Organización no gubernamental,La ONG ha lanzado una campaña de concienciació...,NaN,NaN
1,PCAP,Pliego de cláusulas administrativas particulares,El contrato de obra pública se regirá por el p...,NaN,NaN


In [22]:
class AcronymDetector(dspy.Signature):
    """
    Detecta los acrónimos, abreviaturas y siglas que contenga el texto. 
    
    TEXTO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
    afectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento
    de la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha
    ajustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,
    el PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.
    Sin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.
    ACRONIMOS:IVA, PYME, IRPF, PIB 
    """
    
    TEXTO = dspy.InputField(desc="Texto en español que puede contener o no, acrónimos, siglas y/o abreviaturas que deben detectarse")
    ACRONIMOS = dspy.OutputField(desc="Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusión de varias otras palabras")
    
class AcronymDetectorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(AcronymDetector)

    def forward(self, texto):  
        response = self.generator(TEXTO=texto)    
        return dspy.Prediction(ACRONIMOS=response.ACRONIMOS)

In [34]:
# Crear una instancia del módulo
my_acronym_detector = AcronymDetectorModule()

# Lista para almacenar los acrónimos detectados
acronyms_detected = []

# Detectar acrónimos en cada texto del DataFrame
for index, row in df.iterrows():
    print(f"Processing row {index}: {row['text']}")
    prediction = my_acronym_detector.forward(row['text'])
    print(f"Detected acronyms: {prediction.ACRONIMOS}")
    acronyms_detected.append(prediction.ACRONIMOS)

Processing row 0: La ONG ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. La organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. Gracias al apoyo de voluntarios y donantes, la ONG ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. Su labor ha sido fundamental para sensibilizar a la población sobre la urgencia de tomar medidas para preservar nuestro entorno natural.
Detected acronyms: ONG
Processing row 1: El contrato de obra pública se regirá por el pliego de cláusulas administrativas particulares (PCAP), el cual establece las condiciones específicas que deben cumplir tanto la administración como la empresa contratista. Es fundamental revisar detenidamente el PCAP para garantizar el cumplimiento de todas las obligaciones y evitar posibles conflictos durante la ejecución del proyecto.
Detected acrony

Detected acronyms: MINHAC
Processing row 21: El SPEE es el encargado de gestionar el empleo en España, facilitando la inserción laboral de los ciudadanos. Gracias a sus programas de formación y orientación, el SPEE ayuda a reducir la tasa de desempleo en el país. Además, colabora estrechamente con las empresas para promover la contratación de trabajadores a través de medidas como los incentivos a la contratación. Sin embargo, es importante seguir mejorando la eficacia del SPEE para garantizar un mercado laboral más dinámico y competitivo.
Detected acronyms: SPEE
Processing row 22: La Organización Nacional de Ciegos Españoles (ONCE) es una institución que trabaja para la integración social y laboral de las personas con discapacidad visual. A través de sus programas de formación y empleo, la ONCE ayuda a mejorar la calidad de vida de este colectivo. Además, su lotería solidaria es una fuente importante de financiación para sus proyectos. En resumen, la ONCE es un ejemplo de compromiso y 

In [28]:
acronyms_detected

['ONG',
 'PCAP',
 'OTAN',
 'GHz',
 'FNUPI',
 'UE',
 'FMI (Fondo Monetario Internacional)',
 'OMS',
 'CEIP',
 'No se detectaron acrónimos, siglas o abreviaturas en el texto.',
 'ffcc',
 'ITV',
 'TVE',
 's.a.',
 'No se encontraron acrónimos, siglas o abreviaturas en el texto.',
 'No se detectaron acrónimos, siglas o abreviaturas en el texto.',
 'PYME, PIB',
 'BME, IBEX',
 'CCAA',
 'CNMV',
 'MINHAC',
 'SPEE',
 'ONCE',
 'OCDE, PYME',
 'BOE, AB, BC',
 'AENA',
 'Renfe, COVID-19',
 'RNE',
 'RRHH']

In [35]:
acronyms_detected

['ONG',
 'PCAP',
 'OTAN',
 'GHz',
 'FNUPI',
 'UE',
 'FMI',
 'OMS',
 'CEIP',
 'None',
 'ffcc',
 'ITV',
 'TVE',
 's.a.',
 'N/A',
 '',
 'PYME, PIB',
 'BME, IBEX 35',
 'ccaa, virus',
 'CNMV',
 'MINHAC',
 'SPEE',
 'ONCE',
 'OCDE, PYME',
 'BOE, AB, BC',
 'AENA',
 'Renfe',
 'RNE',
 'RRHH']

In [36]:
# Guardar los acrónimos detectados
df['detected_acronyms_LLaMA'] = acronyms_detected

In [37]:
df

,acronyms,expanded_acronyms,text,detected_acronyms,LLM_expanded_acronyms,detected_acronyms_GPT-3.5,detected_acronyms_LLaMA
0,ONG,Organización no gubernamental,La ONG ha lanzado una campaña de concienciació...,NaN,NaN,ONG,ONG
1,PCAP,Pliego de cláusulas administrativas particulares,El contrato de obra pública se regirá por el p...,NaN,NaN,PCAP,PCAP
2,OTAN,Organización del Tratado del Atlántico Norte,La OTAN es una organización internacional que ...,NaN,NaN,OTAN,OTAN
3,ghz,gigaherzio,El nuevo teléfono móvil cuenta con un procesad...,NaN,NaN,GHz,GHz
4,UNICEF,Fondo de las Naciones Unidas para la Infancia,El FNUPI (Fondo de las Naciones Unidas para la...,NaN,NaN,FNUPI,FNUPI
5,UE,Unión Europea,La Unión Europea (UE) ha implementado nuevas m...,NaN,NaN,UE,UE
6,FMI,Fondo monetario internacional,El FMI (Fondo Monetario Internacional) ha reco...,NaN,NaN,FMI (Fondo Monetario Internacional),FMI
7,OMS,Organización mundial de la salud,La Organización Mundial de la Salud (OMS) ha e...,NaN,NaN,OMS,OMS
8,ceip,Colegio de Educación Infantil y Primaria,El CEIP (Centro de Educación Infantil y Primar...,NaN,NaN,CEIP,CEIP
9,avda,avenida,La avenida principal de la ciudad estaba abarr...,NaN,NaN,"No se detectaron acrónimos, siglas o abreviatu...",None


In [38]:
lm.inspect_history(4)




Detecta los acrónimos, abreviaturas y siglas que contenga el texto. 

TEXTO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
afectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento
de la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha
ajustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,
el PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.
Sin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.
ACRONIMOS:IVA, PYME, IRPF, PIB

---

Follow the following format.

TEXTO: Texto en español que puede contener o no, acrónimos, siglas y/o abreviaturas que deben detectarse
ACRONIMOS: Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusión de varias otras 

'\n\n\nDetecta los acrónimos, abreviaturas y siglas que contenga el texto. \n\nTEXTO: La reforma del sistema tributario ha traído cambios significativos en el IVA,\nafectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento\nde la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha\najustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,\nel PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.\nSin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.\nACRONIMOS:IVA, PYME, IRPF, PIB\n\n---\n\nFollow the following format.\n\nTEXTO: Texto en español que puede contener o no, acrónimos, siglas y/o abreviaturas que deben detectarse\nACRONIMOS: Lista de acrónimos. Un acrónimo, es un tipo de palabra acuñada deliberadamente a partir de la fusi

In [31]:
df['text'].iloc[24]

'El Boletín Oficial del Estado (BOE) publicó hoy una nueva normativa que afecta a las empresas del sector financiero. La Asociación de Bancos (AB) ha expresado su preocupación por las implicaciones de esta medida en sus operaciones diarias. Sin embargo, el Banco Central (BC) ha asegurado que estas regulaciones son necesarias para garantizar la estabilidad del sistema financiero. Los analistas financieros están evaluando el impacto de estas nuevas normas en el mercado y se espera que emitan un informe detallado en los próximos días.'

# Desambiguación de acrónimos con dspy (Etapa 2)

In [ ]:
#Montar base de datos tipo Chroma para hacer el retriever sobre los docs que contengan el acrónimo de la lista.
#Para ello debo normalizar tanto la lista de acrónimos que es una columna del df en tiempo real. Y además normalizar
# el texto de los docs que se guarden en la base de datos.

In [39]:
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()

# Habría que crear un identifier único de los docs en futuras versiones

In [ ]:
'''
import chromadb
from chromadb.utils import embedding_functions
from dspy.retrieve.chromadb_rm import ChromadbRM

# Using HuggingFace models. The embedding function a huggingface api_key
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="HUGGINGFACE_API_KEY", 
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
# Create a new chroma collection
hugging_face_ef = client.get_or_create_collection(name="huggingface_embeddings", embedding_function=huggingface_ef)

chroma_client = client = chromadb.PersistentClient(path="/export/usuarios_ml4ds/cggamella/RAG_tool")

collection = chroma_client.get_or_create_collection(name="db_POC", embedding_function=hugging_face_ef)

collection.add(
    documents=[
        "couch, bed, table, chair", 
        "computer, server, table, chair"],
    metadatas=[
        {"source": "Bedroom"}, 
        {"source": "Office"}
        ],
    ids=[
        "id1", 
        "id2"
    ]
)

rm = ChromadbRM(collection_name='db_POC', persist_directory="/export/usuarios_ml4ds/cggamella/RAG_tool", embedding_function=huggingface_ef)
print(rm('comfy'))
'''

In [72]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
import time

In [74]:
# Crear el text splitter, [chunk_size: #caracteres de cada chunk];
#[chunk_overlap: #caracteres solapan entre chunks para no perder info.]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df.iterrows():
    example_document = row['text']
    print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"}) #, "identifier": row['identifier']})
    print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)

el example doc es: la ong ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. la organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. gracias al apoyo de voluntarios y donantes, la ong ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. su labor ha sido fundamental para sensibilizar a la población sobre la urgencia de tomar medidas para preservar nuestro entorno natural.
El doc es: page_content='la ong ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. la organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. gracias al apoyo de voluntarios y donantes, la ong ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. su labor ha s

In [75]:
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/RAG_tool'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db_POC').as_posix()

In [76]:
path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = api_key

/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env


In [84]:
# Calcular los embeddings con un modelo de uso libre de HuggingFace
# Using HuggingFace models. The embedding function a huggingface api_key
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="HUGGINGFACE_API_KEY", 
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [88]:
start = time.time()

# Almacenar los fragmentos en una base de datos vectorial usando Chroma
# Se extrae el contenido (page_content).El contenido se pasa a OpenAIEmbeddings
# para obtener los embeddings. El vector resultante se almacena en la base de datos junto con los metadatos(índices).
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory 
)

# Tiempo total de ejecución, tarda 51 mins en 300k docs
end = time.time()
print(f"Total time is {end - start} seconds")

Total time is 0.952500581741333 seconds


In [101]:
results = vectordb.get(limit=6, include=['documents', 'embeddings'])

In [102]:
results

{'ids': ['04c7f02a-3307-412b-bab9-697ecce19f2c',
  '09cd7d7c-7d1c-4c93-829c-1d5260b59653',
  '371f3b95-1069-47ed-97a4-de36af891f46',
  '384914cd-0b47-4e67-9157-39bf27b98279',
  '3e557732-d327-4483-acae-1f2bb82092c5',
  '3edb3fe6-7bd4-439a-9b29-6417f2d87e3e'],
 'embeddings': [[-0.2557930648326874,
   0.3257238566875458,
   -0.21083670854568481,
   -0.1321161985397339,
   -0.19688180088996887,
   -0.2639316916465759,
   -0.04668450728058815,
   -0.04649772122502327,
   0.1556236296892166,
   -0.005106822121888399,
   0.08584227412939072,
   0.3596961796283722,
   0.2298348844051361,
   -0.24303963780403137,
   0.12804442644119263,
   -0.023083386942744255,
   0.3545556962490082,
   0.06278876215219498,
   -0.10064169019460678,
   -0.06173530966043472,
   0.022016381844878197,
   -0.3266386091709137,
   0.125695139169693,
   0.039284247905015945,
   -0.14091475307941437,
   0.1257604956626892,
   -0.0682797059416771,
   0.030220599845051765,
   0.09504822641611099,
   -0.15652069449424744

# Custom Retrieval Module

In [96]:
class AcronymAwareRetriever(dspy.Retrieve):
    """
    Custom retriever for searching documents containing a specific acronym.
    """
    def __init__(self, vectordb, k:int = 3):
        super().__init__(k=k)
        self.vectordb = vectordb

    def forward(self, query: Union[str, List[str]], k: int = None) -> dspy.Prediction:
        """
        Retrieve documents containing the provided acronym by directly querying
        the vector database.
        """
        search_query = {"$contains": query}
        results = self.vectordb.get(where_document=search_query, limit=k if k else self.k)
        documents = results['documents']
        return dspy.Prediction(passages=documents)

In [99]:
acronym_retriever = AcronymAwareRetriever(vectordb, k=5)

# Usando el método 'forward' que retorna un objeto Prediction con los documentos
result_prediction = acronym_retriever("ghz")

# Imprimir cada documento
for doc in result_prediction.passages:
    print(doc)
    print('*' * 100)

el nuevo teléfono móvil cuenta con un procesador de última generación que alcanza una velocidad de 2.5 ghz, lo que significa que puede ejecutar aplicaciones de forma más rápida y eficiente. con esta tecnología, los usuarios podrán disfrutar de una experiencia más fluida y sin interrupciones en sus actividades diarias.
****************************************************************************************************


In [106]:
class AcronymExpander(dspy.Signature):
    """
    Expande los acrónimos basándose en el contexto del texto.
    
    TEXTO: Las antenas que operan en la banda de los 60 Ghz se dice que funcionan en la banda de ondas milimétricas
    ACRONIMO: ghz
    EXPANSION: gigahertzio
    """
    
    TEXTO = dspy.InputField(desc="Texto que contiene el acrónimo,sigla o abreviatura y aporta información sobre la forma expandida")
    ACRONIMO = dspy.InputField(desc="Acrónimo que necesita ser expandido")
    EXPANSION = dspy.OutputField(desc="Es la forma expandida del acrónimo")
    
class AcronymExpanderModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.expander = dspy.Predict(AcronymExpander)

    def forward(self, texto, acronimo):  
        response = self.expander(TEXTO=texto, ACRONIMO=acronimo)    
        return dspy.Prediction(EXPANSION=response.EXPANSION)

In [127]:
acronym_retriever = AcronymAwareRetriever(vectordb=vectordb, k=5)
expander_module = AcronymExpanderModule()

# DataFrame para almacenar los resultados
results_df = pd.DataFrame(columns=["acronym", "text", "expanded"])

# Lista para almacenar los resultados
results = []
# Normalizo porque los docs que están en la bbdd están normalizados
acronyms = df['detected_acronyms_LLaMA'].str.lower().tolist()
acronyms = [acronym for acronym in acronyms if acronym not in ['', 'n/a']]

for index, row in df.iterrows():
    text = row['text'] 
    for acronym in acronyms:
        # Recuperar docs relevantes utilizando el retriever
        retrieved_docs = acronym_retriever.forward(acronym).passages
        # Incluir la lógica de que si ya está ese acrónimo desambiguado guardar algún grafo de conocimiento..
        for doc in retrieved_docs:
            print("Los docs retrieved son:",retrieved_docs)
            expansion_result = expander_module.forward(doc, acronym)
            print("El expansion result es:", expansion_result)
            expanded_form = expansion_result.EXPANSION
            print("La forma expandida es:", expanded_form)
            # Añadir a la lista
            results.append({"text": doc, "expanded": expanded_form, "acronym": acronym})

# Guardar los resultados en un dataframe auxiliar
results_df = pd.DataFrame(results)

Los docs retrieved son: ['la ong ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. la organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. gracias al apoyo de voluntarios y donantes, la ong ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. su labor ha sido fundamental para sensibilizar a la población sobre la urgencia de tomar medidas para preservar nuestro entorno natural.', 'el accidente ocurrió en el punto kilométrico 50 de la autopista, causando un gran congestionamiento de tráfico. los servicios de emergencia llegaron rápidamente al lugar para atender a los heridos y restablecer la circulación. la policía investiga las causas del suceso para determinar si hubo alguna negligencia por parte de los conductores involucrados.']
El expansion result es: Prediction(
    EXPANSION='Organización No Gubern

El expansion result es: Prediction(
    EXPANSION='comisión nacional del mercado de valores'
)
La forma expandida es: comisión nacional del mercado de valores
Los docs retrieved son: ['el ministerio de hacienda y administraciones públicas (minhac) ha anunciado nuevas medidas para mejorar la eficiencia en la gestión de los recursos públicos. estas acciones buscan optimizar el uso de los fondos y garantizar una mayor transparencia en el manejo de los mismos. además, se ha implementado un plan de formación para los funcionarios con el objetivo de fortalecer sus capacidades en materia financiera. sin duda, estas iniciativas son clave para impulsar el desarrollo económico y social del país.']
El expansion result es: Prediction(
    EXPANSION='Ministerio de Hacienda y Administraciones Públicas'
)
La forma expandida es: Ministerio de Hacienda y Administraciones Públicas
Los docs retrieved son: ['el spee es el encargado de gestionar el empleo en españa, facilitando la inserción laboral de los 

El expansion result es: Prediction(
    EXPANSION='unidades electrónicas'
)
La forma expandida es: unidades electrónicas
Los docs retrieved son: ['el nuevo teléfono móvil cuenta con un procesador de última generación que alcanza una velocidad de 2.5 ghz, lo que significa que puede ejecutar aplicaciones de forma más rápida y eficiente. con esta tecnología, los usuarios podrán disfrutar de una experiencia más fluida y sin interrupciones en sus actividades diarias.', 'la unión europea (ue) ha implementado nuevas medidas para fortalecer la cooperación entre sus países miembros. la libre circulación de bienes, servicios, personas y capitales es uno de los pilares fundamentales de la ue. además, la unión europea ha establecido un fondo de recuperación para apoyar a las economías más afectadas por la crisis actual. la integración europea sigue siendo un objetivo clave para garantizar la estabilidad y el progreso en la región.', 'el contrato de obra pública se regirá por el pliego de cláusulas

Los docs retrieved son: ['el transporte público en la ciudad se ha visto afectado por el cierre temporal de las estaciones de ffcc debido a trabajos de mantenimiento. los usuarios han expresado su malestar por la falta de información sobre las alternativas de transporte disponibles. la empresa responsable de las ffcc ha prometido una pronta solución para minimizar las molestias a los pasajeros.']
El expansion result es: Prediction(
    EXPANSION='Ferrocarriles de la Ciudad de Córdoba (en este caso, se refiere a un sistema de transporte público en particular) o Ferrocarril Federal de la Ciudad de Córdoba. (en este caso, se refiere a un sistema de transporte público en particular) o Ferrocarril Federal de la Ciudad de Córdoba. (en'
)
La forma expandida es: Ferrocarriles de la Ciudad de Córdoba (en este caso, se refiere a un sistema de transporte público en particular) o Ferrocarril Federal de la Ciudad de Córdoba. (en este caso, se refiere a un sistema de transporte público en particular

Los docs retrieved son: ['la comisión nacional del mercado de valores (cnmv) es el organismo encargado de supervisar e inspeccionar los mercados de valores en españa. su principal objetivo es garantizar la transparencia y la protección de los inversores, velando por el correcto funcionamiento de los mercados financieros. la cnmv se encarga de regular la actividad de las entidades financieras y de velar por el cumplimiento de la normativa vigente en materia de valores. su labor es fundamental para mantener la confianza en el sistema financiero y para prevenir posibles abusos o fraudes en el mercado de valores.']
El expansion result es: Prediction(
    EXPANSION='comisión nacional del mercado de valores'
)
La forma expandida es: comisión nacional del mercado de valores
Los docs retrieved son: ['el ministerio de hacienda y administraciones públicas (minhac) ha anunciado nuevas medidas para mejorar la eficiencia en la gestión de los recursos públicos. estas acciones buscan optimizar el uso

...

Los docs retrieved son: ['la empresa de transporte ferroviario renfe ha anunciado la incorporación de trenes de alta velocidad en varias rutas, con el objetivo de mejorar la conectividad entre ciudades y reducir los tiempos de viaje. esta medida forma parte de su plan de modernización y expansión, que busca aumentar la satisfacción de los usuarios y fomentar el uso del transporte público. además, renfe ha implementado nuevas medidas de seguridad y limpieza en sus estaciones y trenes, en respuesta a la pandemia de covid-19.']
El expansion result es: Prediction(
    EXPANSION='red nacional de ferrocarriles españoles'
)
La forma expandida es: red nacional de ferrocarriles españoles
Los docs retrieved son: ['la emisora de radio nacional españa ha lanzado una nueva campaña para promover la diversidad cultural en sus programas. con el objetivo de llegar a un público más amplio, la rne ha incorporado contenidos variados que abarcan desde música tradicional hasta debates políticos. esta estrat

Los docs retrieved son: ['el contrato de obra pública se regirá por el pliego de cláusulas administrativas particulares (pcap), el cual establece las condiciones específicas que deben cumplir tanto la administración como la empresa contratista. es fundamental revisar detenidamente el pcap para garantizar el cumplimiento de todas las obligaciones y evitar posibles conflictos durante la ejecución del proyecto.']
El expansion result es: Prediction(
    EXPANSION='pliego de cláusulas administrativas particulares'
)
La forma expandida es: pliego de cláusulas administrativas particulares
Los docs retrieved son: ['la otan es una organización internacional que tiene como objetivo principal garantizar la seguridad y defensa de sus países miembros. desde su creación, la otan ha desempeñado un papel crucial en la estabilidad y la paz mundial, colaborando estrechamente en misiones de mantenimiento de la paz y en la lucha contra el terrorismo. gracias a la cooperación entre los países miembros de l

Los docs retrieved son: ['el fmi (fondo monetario internacional) ha recomendado al gobierno implementar medidas de austeridad para reducir el déficit fiscal. esta propuesta ha generado controversia entre los ciudadanos, quienes temen que las políticas de ajuste afecten negativamente a la economía. sin embargo, algunos expertos señalan que las recomendaciones del fmi son necesarias para garantizar la estabilidad financiera del país a largo plazo.']
El expansion result es: Prediction(
    EXPANSION='fondo monetario internacional'
)
La forma expandida es: fondo monetario internacional
Los docs retrieved son: ['la organización mundial de la salud (oms) ha emitido nuevas recomendaciones para combatir la propagación del virus. es importante seguir las directrices de la oms para proteger la salud de la población y evitar un aumento en los casos de contagio.']
El expansion result es: Prediction(
    EXPANSION='Organización Mundial de la Salud'
)
La forma expandida es: Organización Mundial de l

Los docs retrieved son: ['la empresa ha decidido cambiar su estructura legal y convertirse en una sociedad anónima (s.a.), con el objetivo de atraer más inversores y expandir sus operaciones a nivel internacional. esta decisión ha sido bien recibida por los accionistas, quienes ven en esta transformación una oportunidad para aumentar su participación en la compañía y diversificar sus inversiones. sin embargo, el proceso de conversión a s.a. requiere cumplir con una serie de requisitos legales y fiscales, por lo que se espera que el cambio se complete en los próximos meses.']
El expansion result es: Prediction(
    EXPANSION='sociedad anónima'
)
La forma expandida es: sociedad anónima
Los docs retrieved son: ['la comisión nacional del mercado de valores (cnmv) es el organismo encargado de supervisar e inspeccionar los mercados de valores en españa. su principal objetivo es garantizar la transparencia y la protección de los inversores, velando por el correcto funcionamiento de los mercad

El expansion result es: Prediction(
    EXPANSION='recursos humanos y relaciones laborales'
)
La forma expandida es: recursos humanos y relaciones laborales
Los docs retrieved son: ['la ong ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. la organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. gracias al apoyo de voluntarios y donantes, la ong ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país. su labor ha sido fundamental para sensibilizar a la población sobre la urgencia de tomar medidas para preservar nuestro entorno natural.', 'el accidente ocurrió en el punto kilométrico 50 de la autopista, causando un gran congestionamiento de tráfico. los servicios de emergencia llegaron rápidamente al lugar para atender a los heridos y restablecer la circulación. la policía investiga las causas del suceso para